## 모델 인자 설정

In [1]:
batch_size=16
epochs=10  #default 300
weights= 'yolov5s.pt'
#evolve = 가중치 진화 #default 300
train_dir= '/home/appuser/animaldata_62_mu/input/train/백내장/'
class_num=2 #nc
class_name= ['무', '유'] #names
resume= False # help='resume most recent training'
imgsz=640
#noval=   help='disable AutoAnchor'
#nosave=  help='resume most recent training'
workers=8  #help='max dataloader workers (per RANK in DDP mode) #CPU 코어  #간단하게 설명하면 데이터 로드할 때 몇개의 프로세스를 사용할래요? 라는 파라미터 이다.
#위에 사이트는 num_workers를 몇으로해야 좋을지에 대한 토의를 하는 포스트인데 실험상 가장 좋았던 공식은 num_workers = 4 * num_of_gpus 라고 한다.
freeze=[0] #동결 없음
lr0 = 0.01 #default 0.01 # initial learning rate (SGD=1E-2, Adam=1E-3)
warmup_bias_lr = 0.1  # warmup initial bias lr
momentum =  0.937  # SGD momentum/Adam beta1
optimizer_type = 'SGD' #choices=['SGD', 'Adam', 'AdamW']
weight_decay = 0.0005  # optimizer weight decay 5e-4
lrf= 0.2  # final OneCycleLR learning rate (lr0 * lrf)
patience = 100

In [5]:
import os
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))
print("LOCAL_RANK", LOCAL_RANK, "RANK", RANK, "WORLD_SIZE", WORLD_SIZE)
last, best ='last.pt', 'best.pt'

LOCAL_RANK -1 RANK -1 WORLD_SIZE 1


In [6]:
import torch
cuda = torch.cuda.is_available()
print(cuda)

True


In [8]:
import torch
device =  torch.device('cuda:0' if cuda else 'cpu')

In [9]:
import random
import numpy as np
def init_seeds(seed=0):
    # Initialize random number generator (RNG) seeds https://pytorch.org/docs/stable/notes/randomness.html
    # cudnn seed 0 settings are slower and more reproducible, else faster and less reproducible
    import torch.backends.cudnn as cudnn
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    cudnn.benchmark, cudnn.deterministic = (False, True) if seed == 0 else (True, False)

In [10]:
init_seeds(1 + RANK)

## 모델 불러오기

In [11]:
models = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=False,  classes=2) # create

# models.yolo.Detect [80, [[10, 13, 16, 30, 33, 23], . . .  [128, 256, 512]] ---> [2, [[10, 13, 16, 30, 33, 23], . . .  [128, 256, 512]]

Using cache found in /home/appuser/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-18 torch 1.10.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3

In [12]:
#freeze
freeze_ = [f'model.{x}.' for x in (freeze if len(freeze) > 1 else range(freeze[0]))] 
for k, v in models.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze_):
            print(f'freezing {k}')
            v.requires_grad = False

In [13]:
int(models.stride.max())

32

In [ ]:
# Image size - 해당 모델의 max stride는 32로,resize 이미지가 32의 배수여야 함

## optimizer, learning rate 설정

In [15]:
# optimizer parameter groups
#pretrained =False
import torch.nn as nn
g0, g1, g2 = [], [], []  
for v in models.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
        g1.append(v.weight)

In [16]:
#optimizer
from torch.optim import SGD, Adam, AdamW, lr_scheduler
if optimizer_type == 'Adam':
    optimizer = Adam(g0, lr=lr0, betas=(momentum, 0.999))  # adjust beta1 to momentum
elif optimizer_type == 'AdamW':
    optimizer = AdamW(g0, lr=lr0, betas=(momentum, 0.999))  # adjust beta1 to momentum
else:
    optimizer = SGD(g0, lr=lr0, momentum=momentum, nesterov=True)

In [17]:
# optimizer parameter group에 값을 이미 넣은 경우 해당 셀은 오류 메세지를 띄울 수 있음(some parameters appear in more than one parameter group)
optimizer.add_param_group({'params': g1, 'weight_decay': weight_decay})  # add g1 with weight_decay
optimizer.add_param_group({'params': g2})  # add g2 (biases)
print(f"{'optimizer:'} {type(optimizer).__name__} with parameter groups "
            f"{len(g0)} weight (no decay), {len(g1)} weight, {len(g2)} bias")
del g0, g1, g2

optimizer: SGD with parameter groups 57 weight (no decay), 60 weight, 60 bias


In [18]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.937
    nesterov: True
    weight_decay: 0

Parameter Group 1
    dampening: 0
    lr: 0.01
    momentum: 0.937
    nesterov: True
    weight_decay: 0.0005

Parameter Group 2
    dampening: 0
    lr: 0.01
    momentum: 0.937
    nesterov: True
    weight_decay: 0
)

In [19]:
# Scheduler
linear_lr = True
if linear_lr:
    lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - lrf) + lrf  # linear
else:
    lf = one_cycle(1, lrf, epochs)  # cosine 1->hyp['lrf']
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)  # plot_lr_scheduler(optimizer, scheduler, epochs)

## 지수 이동 평균 적용

In [20]:
from copy import deepcopy
class ModelEMA:
    """ Updated Exponential Moving Average (EMA) from https://github.com/rwightman/pytorch-image-models
    Keeps a moving average of everything in the model state_dict (parameters and buffers)
    For EMA details see https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    """

    def __init__(self, model, decay=0.9999, updates=0):
        # Create EMA
        self.ema = deepcopy(model.module if type(model) in (nn.parallel.DataParallel, nn.parallel.DistributedDataParallel) else model).eval()  # FP32 EMA
        # if next(model.parameters()).device.type != 'cpu':
        #     self.ema.half()  # FP16 EMA
        self.updates = updates  # number of EMA updates
        self.decay = lambda x: decay * (1 - math.exp(-x / 2000))  # decay exponential ramp (to help early epochs)
        for p in self.ema.parameters():
            p.requires_grad_(False)

    def update(self, model):
        # Update EMA parameters
        with torch.no_grad():
            self.updates += 1
            d = self.decay(self.updates)

            msd = de_parallel(model).state_dict()  # model state_dict
            for k, v in self.ema.state_dict().items():
                if v.dtype.is_floating_point:
                    v *= d
                    v += (1 - d) * msd[k].detach()

    def update_attr(self, model, include=(), exclude=('process_group', 'reducer')):
        # Update EMA attributes
        copy_attr(self.ema, model, include, exclude)

In [21]:
# EMA
ema = ModelEMA(models) if RANK in [-1, 0] else None
#지수 이동 평균 (EMA)은 가장 최근의 데이터 포인트에 더 큰 가중치와 중요성을 부여하는 이동 평균 ( MA ) 유형입니다. 지수 이동 평균은 지수 가중치 이동 평균 이라고도 합니다. 지수 가중 이동 평균은 해당 기간의 모든 관측치에 동일한 가중치를 적용하는 단순 이동 평균 ( SMA ) 보다 최근 가격 변동에 더 크게 반응합니다.

## DataParallel 적용

In [23]:
# DP mode
if cuda and RANK == -1 and torch.cuda.device_count() > 1: # True
    print('WARNING: DP not recommended, use torch.distributed.run for best DDP Multi-GPU results.\n'
                   'See Multi-GPU Tutorial at https://github.com/ultralytics/yolov5/issues/475 to get started.')
    model = torch.nn.DataParallel(models)
#DataParallel은 단일 머신에서만 가능, 속도도 느림

See Multi-GPU Tutorial at https://github.com/ultralytics/yolov5/issues/475 to get started.


## trainloader

In [25]:
root = '/home/appuser/animaldata_62_mu/input'
disease_code = '백내장_초음파'
split_ratio = 0.2
img_resize = 640
mode = "train" 

In [26]:
"""Dataset.py"""
import cv2
from torch.utils.data import Dataset, dataloader, distributed # #배치에 대해서 반복하기 편리하게 해줍니다.
from IPython.display import Image
import matplotlib.pyplot as plt
import os
import matplotlib.image as img
import json
plt.rcParams["figure.figsize"] = (5,5)

class PETDataset_eye(Dataset): 
    
    def __init__(self, root,disease_code ,split_ratio, mode, img_resize) : # file_list # imgsz, stride=int(stride)
        self.root = root
        self.disease_code = disease_code
        self.mode = mode
        self.split_ratio = split_ratio        
        self.dir = {"train": self.root + '/' + "train" + '/' + self.disease_code,
                    "test": self.root + '/' + "test" + '/' + self.disease_code}
        self.file_list = []
        self.img_resize= img_resize
                
        if self.mode == "train" : 
            train_무_images = [f"{self.dir['train']}/무/" + f for f in os.listdir(f"{self.dir['train']}/무") if (f[-3:]=="jpg")]
            train_유_images = [f"{self.dir['train']}/유/" + f for f in os.listdir(f"{self.dir['train']}/유") if (f[-3:]=="jpg")]
            self.file_list = train_무_images[:int(len(train_무_images) * (1 - self.split_ratio))] + \
                                train_유_images[:int(len(train_유_images) * (1 - self.split_ratio))]
            

        elif self.mode == "valid" : 
            train_무_images = [f"{self.dir['train']}/무/" + f for f in os.listdir(f"{self.dir['train']}/무") if (f[-3:]=="jpg")]
            train_유_images = [f"{self.dir['train']}/유/" + f for f in os.listdir(f"{self.dir['train']}/유") if (f[-3:]=="jpg")]
            self.file_list = train_무_images[int(len(train_무_images) * (1 - self.split_ratio)):] + \
                              train_유_images[int(len(train_유_images) * (1 - self.split_ratio)):]
    
        else : #mode = "test"
            test_무_images = [f"{self.dir['test']}/무/" + f for f in os.listdir(f"{self.dir['test']}/무") if (f[-3:]=="jpg")]
            test_유_images = [f"{self.dir['test']}/유/" + f for f in os.listdir(f"{self.dir['test']}/유") if (f[-3:]=="jpg")]
            self.file_list = test_무_images + test_유_images           
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index): # model input, target for loss from file_list
        img_path = self.file_list[index]
        img = torch.from_numpy(self.resize_img(img_path))
#         plt.imshow(img)
#         plt.show()
        json_path = os.path.join(img_path.replace(".jpg", ".json"))    
        with open(json_path) as f :
            anno=json.load(f)
        anno_label = anno['label']
        label = [0,0,self.img_resize,self.img_resize]  #XYXY
        if anno['label']['label_disease_nm'] != "백내장" :
            assert true, f"잘못된 데이터 {anno['label']['label_disease_nm']} 질환이 들어왔습니다."
        if anno['label']['label_disease_lv_3'] == "무" : 
            label.append(0)
        elif anno['label']['label_disease_lv_3'] == "유" : 
            label.append(1)
        else :
            assert true, "anno['label']['label_disease_lv_3'] 이 잘못되었습니다."
        
        return {'input': img, 'target': torch.Tensor(label)} # 하나 이미지에 대한 정의
    
    def resize_img(self, image_path) :   # stride 추가하기 근데 뭘 추가해야할려나.
        im = cv2.imread(image_path)  # BGR
        plt.imshow(im)
        plt.show()
        h0, w0 = im.shape[:2]
        r = self.img_resize / max(h0, w0)  # ratio
        if r != 1:  # if sizes are not equal
            im_2 = cv2.resize(im,(int(w0 * r), int(h0 * r)),
                interpolation=cv2.INTER_LINEAR if r > 1 else cv2.INTER_AREA)
        return im_2

In [27]:
# 이미지 크기 (400,400,3)가 아닌 데이터 목록
pet_for_wh = PETDataset_eye(root,disease_code,split_ratio, "train", img_resize)
for image_path in pet_for_wh.file_list :
    im = cv2.imread(image_path) 
    h0, w0 = im.shape[:2]
    #print(image_path, h0, w0)
    if h0 != 400 or w0 != 400 :
        print("wrong", image_path, h0,w0)  # 없음

In [28]:
# 이미지 크기 (400,400,3)가 아닌 데이터 목록
pet_for_wh_2 = PETDataset_eye(root,disease_code,split_ratio, "test", img_resize)
for image_path in pet_for_wh_2.file_list :
    im = cv2.imread(image_path) 
    h0, w0 = im.shape[:2]
    #print(image_path, h0, w0)
    if h0 != 400 or w0 != 400 :
        print("wrong", image_path, h0,w0) #없음

In [29]:
pet = PETDataset_eye(root,disease_code,split_ratio, mode, img_resize)
pet.__len__()
pet.file_list[0]
pet[99]

{'input': tensor([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         ...,
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
 
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=torch.uint8),
 'target': tensor([  0.,   0., 640., 640.,   0.])}

In [37]:
from contextlib import contextmanager
@contextmanager
def torch_distributed_zero_first(local_rank: int):
    """
    Decorator to make all processes in distributed training wait for each local_master to do something.
    """
    if local_rank not in [-1, 0]:
        dist.barrier(device_ids=[local_rank])
    yield
    if local_rank == 0:
        dist.barrier(device_ids=[0])
        
def create_dataloader(path, imgsz, batch_size, mode, rank=-1, workers=8, shuffle=False):
    with torch_distributed_zero_first(rank):  # init dataset *.cache only once if DDP
#        dataset = LoadImagesAndLabels(path, imgsz, batch_size, stride=int(stride), pad=pad)
#             0.path 가 존재하는지 확인
#             1. cache 을 load 해서 이미지 수 found, missing, empty, corrupt, total 읽어오는 듯함 
#             2.라벨 지정한 라벨만 읽어오는 듯함
#             3. 빠른 학습을 위해 이미지를 cache 함.
        dataset = PETDataset_eye(root,disease_code,split_ratio, mode, img_resize)
        
    batch_size = min(batch_size, len(dataset))
    nd = torch.cuda.device_count()  # number of CUDA devices
    nw = min([os.cpu_count() // max(nd, 1), batch_size if batch_size > 1 else 0, workers])  # number of workers
    #os.cpu_count() =16
    sampler = None if rank == -1 else distributed.DistributedSampler(dataset, shuffle=shuffle)
    print("batch_size", batch_size,"number of CUDA devices", nd, "number of workers", nw )
    
    return InfiniteDataLoader(dataset, #Dataloader 도 가능
                  batch_size=batch_size,
                  shuffle=shuffle and sampler is None,
                  num_workers=nw,
                  sampler=sampler,
                  pin_memory=True,
                  collate_fn=None), dataset

class InfiniteDataLoader(dataloader.DataLoader):
    """ Dataloader that reuses workers
    Uses same syntax as vanilla DataLoader
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        object.__setattr__(self, 'batch_sampler', _RepeatSampler(self.batch_sampler))
        self.iterator = super().__iter__()

    def __len__(self):
        return len(self.batch_sampler.sampler)

    def __iter__(self):
        for i in range(len(self)):
            yield next(self.iterator) 
            
class _RepeatSampler:
    """ Sampler that repeats forever
    Args:
        sampler (Sampler)
    """

    def __init__(self, sampler):
        self.sampler = sampler

    def __iter__(self):
        while True:
            yield from iter(self.sampler)

In [38]:
# Trainloader
train_loader, dataset = create_dataloader(train_dir, imgsz, batch_size // WORLD_SIZE, mode="train",  rank=LOCAL_RANK, workers=workers, shuffle=True)
#mlc = int(np.concatenate(dataset.labels, 0)[:, 0].max())  # max label class
nb = len(train_loader)  # number of batches
print( "number of batches", nb ) ##print("max label class", mlc, "number of batches", nb )
# assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data}. Possible class labels are 0-{nc - 1}'

batch_size 16 number of CUDA devices 2 number of workers 8
number of batches 688


In [39]:
# Validloader
if RANK in [-1, 0]:
    val_loader = create_dataloader(train_dir, imgsz, batch_size // WORLD_SIZE, mode="valid",  rank=LOCAL_RANK, workers=workers, shuffle=True)

batch_size 16 number of CUDA devices 2 number of workers 8


In [40]:
start_epoch, best_fitness = 0, 0.0

In [ ]:
# torch.nn.DataParallel(models) 사용시 "model.module", dataparallel 사용안할시 "model.model"

## 훈련(Train)

In [41]:
class EarlyStopping:
    # YOLOv5 simple early stopper
    def __init__(self, patience=30):
        self.best_fitness = 0.0  # i.e. mAP
        self.best_epoch = 0
        self.patience = patience or float('inf')  # epochs to wait after fitness stops improving to stop
        self.possible_stop = False  # possible stop may occur next epoch

    def __call__(self, epoch, fitness):
        if fitness >= self.best_fitness:  # >= 0 to allow for early zero-fitness stage of training
            self.best_epoch = epoch
            self.best_fitness = fitness
        delta = epoch - self.best_epoch  # epochs without improvement
        self.possible_stop = delta >= (self.patience - 1)  # possible stop may occur next epoch
        stop = delta >= self.patience  # stop training if patience exceeded
        if stop:
            LOGGER.info(f'Stopping training early as no improvement observed in last {self.patience} epochs. '
                        f'Best results observed at epoch {self.best_epoch}, best model saved as best.pt.\n'
                        f'To update EarlyStopping(patience={self.patience}) pass a new patience value, '
                        f'i.e. `python train.py --patience 300` or use `--patience 0` to disable EarlyStopping.')
        return stop

In [35]:
# Start training
import time
from torch.cuda import amp
t0 = time.time()
nw = max(round(3.0  * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
last_opt_step = -1
maps = np.zeros(class_num)  # mAP per class
results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
scheduler.last_epoch = start_epoch - 1  # do not move
scaler = amp.GradScaler(enabled=cuda)
stopper = EarlyStopping(patience=patience)
print(f'Image sizes train {imgsz} ,  val {imgsz}\n'
            f'Using {train_loader.num_workers * WORLD_SIZE} dataloader workers\n'
            f'Starting training for {epochs} epochs...')

Image sizes train 640 ,  val 640
Using 8 dataloader workers
Starting training for 10 epochs...


In [46]:
for j, x in enumerate(optimizer.param_groups):
    if 'momentum' in x:
        print(x['momentum'])

0.8456002906976745
0.8456002906976745
0.8456002906976745


In [45]:
from tqdm import tqdm
nbs = 64  # nominal batch size
#for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
for epoch in range(start_epoch, 1):
    model.train()
    mloss = torch.zeros(3, device=device)  # mean losses
    if RANK != -1:
        train_loader.sampler.set_epoch(epoch)
    pbar = enumerate(train_loader)
    print(('\n' + '%10s' * 7) % ('Epoch', 'gpu_mem', 'box', 'obj', 'cls', 'labels', 'img_size'))
    if RANK in [-1, 0]:
        pbar = tqdm(pbar, total=nb, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
    optimizer.zero_grad()
    for i, value in enumerate(train_loader):  # batch -------------------------------------------------------------
        ni = i + nb * epoch  # number integrated batches (since train start)
        #imgs = imgs.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
        imgs= value["input"]
        # Warmup
        if ni <= nw:
            xi = [0, nw]  # x interp
            # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
            accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
            for j, x in enumerate(optimizer.param_groups):
                # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                x['lr'] = np.interp(ni, xi, [warmup_bias_lr if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                if 'momentum' in x:
                    x['momentum'] = np.interp(ni, xi, [ 0.8, 0.937])
        """여기까지 진행함"""

        # Forward
        with amp.autocast(enabled=cuda):
            pred = model(imgs)  # forward
            loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size 
            if RANK != -1:
                loss *= WORLD_SIZE  # gradient averaged between devices in DDP mode
            if opt.quad:
                loss *= 4.

        # Backward
        scaler.scale(loss).backward()

        # Optimize
        if ni - last_opt_step >= accumulate:
            scaler.step(optimizer)  # optimizer.step
            scaler.update()
            optimizer.zero_grad()
            if ema:
                ema.update(model)
            last_opt_step = ni                    

  0%|          | 0/688 [00:04<?, ?it/s]



     Epoch   gpu_mem       box       obj       cls    labels  img_size


  0%|          | 0/688 [00:00<?, ?it/s]

In [48]:
compute_loss = ComputeLoss(model)  # init loss class

AttributeError: 'DataParallel' object has no attribute 'hyp'

AttributeError: 'AutoShape' object has no attribute 'hyp'

In [57]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('module',
               AutoShape(
                 (model): Model(
                   (model): Sequential(
                     (0): Conv(
                       (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
                       (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
                       (act): SiLU(inplace=True)
                     )
                     (1): Conv(
                       (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                       (bn): BatchNorm2d(64, eps=0.001, momen

In [47]:
class ComputeLoss:
    # Compute losses
    def __init__(self, model, autobalance=False):
        self.sort_obj_iou = False
        device = next(model.parameters()).device  # get model device
        h = model.hyp  # hyperparameters

        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['cls_pw']], device=device))
        BCEobj = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['obj_pw']], device=device))

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = smooth_BCE(eps=h.get('label_smoothing', 0.0))  # positive, negative BCE targets

        # Focal loss
        g = h['fl_gamma']  # focal loss gamma
        if g > 0:
            BCEcls, BCEobj = FocalLoss(BCEcls, g), FocalLoss(BCEobj, g)

        det = de_parallel(model).model[-1]  # Detect() module
        self.balance = {3: [4.0, 1.0, 0.4]}.get(det.nl, [4.0, 1.0, 0.25, 0.06, 0.02])  # P3-P7
        self.ssi = list(det.stride).index(16) if autobalance else 0  # stride 16 index
        self.BCEcls, self.BCEobj, self.gr, self.hyp, self.autobalance = BCEcls, BCEobj, 1.0, h, autobalance
        for k in 'na', 'nc', 'nl', 'anchors':
            setattr(self, k, getattr(det, k))

    def __call__(self, p, targets):  # predictions, targets, model
        device = targets.device
        lcls, lbox, lobj = torch.zeros(1, device=device), torch.zeros(1, device=device), torch.zeros(1, device=device)
        tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

        # Losses
        for i, pi in enumerate(p):  # layer index, layer predictions
            b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
            tobj = torch.zeros_like(pi[..., 0], device=device)  # target obj

            n = b.shape[0]  # number of targets
            if n:
                ps = pi[b, a, gj, gi]  # prediction subset corresponding to targets

                # Regression
                pxy = ps[:, :2].sigmoid() * 2 - 0.5
                pwh = (ps[:, 2:4].sigmoid() * 2) ** 2 * anchors[i]
                pbox = torch.cat((pxy, pwh), 1)  # predicted box
                iou = bbox_iou(pbox.T, tbox[i], x1y1x2y2=False, CIoU=True)  # iou(prediction, target)
                lbox += (1.0 - iou).mean()  # iou loss

                # Objectness
                score_iou = iou.detach().clamp(0).type(tobj.dtype)
                if self.sort_obj_iou:
                    sort_id = torch.argsort(score_iou)
                    b, a, gj, gi, score_iou = b[sort_id], a[sort_id], gj[sort_id], gi[sort_id], score_iou[sort_id]
                tobj[b, a, gj, gi] = (1.0 - self.gr) + self.gr * score_iou  # iou ratio

                # Classification
                if self.nc > 1:  # cls loss (only if multiple classes)
                    t = torch.full_like(ps[:, 5:], self.cn, device=device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    lcls += self.BCEcls(ps[:, 5:], t)  # BCE

                # Append targets to text file
                # with open('targets.txt', 'a') as file:
                #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

            obji = self.BCEobj(pi[..., 4], tobj)
            lobj += obji * self.balance[i]  # obj loss
            if self.autobalance:
                self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

        if self.autobalance:
            self.balance = [x / self.balance[self.ssi] for x in self.balance]
        lbox *= self.hyp['box']
        lobj *= self.hyp['obj']
        lcls *= self.hyp['cls']
        bs = tobj.shape[0]  # batch size

        return (lbox + lobj + lcls) * bs, torch.cat((lbox, lobj, lcls)).detach()

In [113]:
models.__dict__ #오,,, 스스로 이 함수 써먹어볼까 떠올렸어><
# class_weights를 yaml에 넣으면 작동할 수 있을지도..?

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('model',
               Model(
                 (model): Sequential(
                   (0): Conv(
                     (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
                     (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
                     (act): SiLU(inplace=True)
                   )
                   (1): Conv(
                     (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                     (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       

In [119]:
models.yaml["nc"]

2